<a href="https://colab.research.google.com/github/KenzaB27/TransUnet/blob/vit/data_processing/create_tfds_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import importlib
import yaml
import cv2 
import os

from google.colab import auth
from pathlib import Path
from tqdm import tqdm 

## Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'PATH_TO_GCP_CREDENTIALS' ## TODO set the path to gcp credentials
!echo $GOOGLE_APPLICATION_CREDENTIALS

project='dd2424-308314' # change to your project name here
os.environ['GCP_PROJECT'] = project 
os.environ['GCP_ACCOUNT'] = 'trans-unet@' + project + '.iam.gserviceaccount.com'

!gcloud auth activate-service-account "$GCP_ACCOUNT" --key-file="$GOOGLE_APPLICATION_CREDENTIALS" --project="$GCP_PROJECT"

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir DataSet
!gcsfuse --implicit-dirs aga_bucket DataSet

In [ ]:
!git clone https://github.com/KenzaB27/TransUnet.git
%cd TransUnet

## Write TfRecords in cloud bucket

In [ ]:
%cd /content/TransUnet/

In [ ]:
import data_processing.data_parser as dp
importlib.reload(dp)
dw = dp.DataWriter('/content/DataSet/train_npz/', '/content/DataSet/synapse-train-224/', height=224, width=224, batch_size=130)

In [ ]:
dw.write_batch_tfrecords()

## Test Loading from tfrecords

In [ ]:
%cd /content/TransUnet/
!git checkout vit
!git pull

In [ ]:
import data_processing.data_parser as dp
importlib.reload(dp)
dr = dp.DataReader("/content/DataSet/synapse-train-224/")
dataset = dr.get_dataset()

In [ ]:
for sample in dataset.take(10):
    print(sample[0].shape)